In [5]:
!pip install --upgrade pip
!pip install mysql-connector-python

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\kiran\Documents\work\db-capstone-project\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import mysql.connector as connector

connection = connector.connect(
    user="root",
    password="killthemall",
    database="LittleLemonDB"
)

In [7]:
cursor = connection.cursor()

connection.database

'littlelemondb'

In [8]:
cursor.execute("""CREATE VIEW OrdersView AS
SELECT OrderID, Quantity, TotalCost
FROM orders
WHERE Quantity > 2""")

ProgrammingError: 1050 (42S01): Table 'OrdersView' already exists

In [9]:
show_tables_query = """SHOW tables"""
cursor.execute(show_tables_query)
results = cursor.fetchall()
cols = cursor.column_names[1:]
print("Tables in the LittleLemonDB database:")
print("--------------------------------------")
print(cols)
for result in results:
    print(result)

Tables in the LittleLemonDB database:
--------------------------------------
()
('bookings',)
('customerdetails',)
('menu',)
('order_delivery_status',)
('orders',)
('ordersview',)
('staff_information',)


In [10]:
orders_query = """SELECT * FROM customerdetails""" 
cursor.execute(orders_query)
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

('CustomerID', 'Contacts', 'Names')
(1, 'john.doe@example.com', 'John Doe')
(2, 'jane.doe@example.com', 'Jane Doe')
(3, 'alice@example.com', 'Alice')
(4, 'bob@example.com', 'Bob')
(5, 'charlie@example.com', 'Charlie')
(6, 'david@example.com', 'David')
(7, 'emily@example.com', 'Emily')
(8, 'frank@example.com', 'Frank')
(9, 'grace@example.com', 'Grace')
(10, 'hannah@example.com', 'Hannah')


In [11]:
orders_query = """SELECT customers.CustomerID, customers.Names, orders.OrderID, orders.TotalCost, menu.Starters, menu.Cuisines, menu.Courses, menu.Drinks, menu.Desserts
FROM customerdetails customers INNER JOIN orders
ON customers.CustomerID = orders.customerID
INNER JOIN menu ON orders.OrderID = menu.OrderID 
WHERE orders.TotalCost > 150  ORDER BY orders.TotalCost""" 
cursor.execute(orders_query)
results = cursor.fetchall()
cols = cursor.column_names
print(cols)
for result in results:
    print(result)

('CustomerID', 'Names', 'OrderID', 'TotalCost', 'Starters', 'Cuisines', 'Courses', 'Drinks', 'Desserts')
(7, 'Emily', 4, 199, 'Chicken Wings', 'American', 'Appetizer', 'Soda', 'Chocolate Cake')
(3, 'Alice', 9, 199, 'Greek Salad', 'Greek', 'Salad', 'Ouzo', 'Baklava')
(8, 'Frank', 2, 295, 'Spring Rolls', 'Chinese', 'Appetizer', 'Green Tea', 'Fruit Tart')
(9, 'Grace', 6, 295, 'Sushi Rolls', 'Japanese', 'Appetizer', 'Sake', 'Mochi Ice Cream')
(5, 'Charlie', 1, 499, 'Garlic Butter Shrimp', 'Italian', 'Appetizer', 'White Wine', 'Tiramisu')
(2, 'Jane Doe', 7, 499, 'Hummus with Pita', 'Lebanese', 'Appetizer', 'Mint Tea', 'Baklava')
(4, 'Bob', 3, 599, 'Caprese Salad', 'Italian', 'Salad', 'Iced Tea', 'Cheesecake')
(6, 'David', 8, 599, 'Tandoori Chicken', 'Indian', 'Main Course', 'Lassi', 'Gulab Jamun')
(1, 'John Doe', 5, 795, 'Tomato Soup', 'French', 'Soup', 'Red Wine', 'Creme Brulee')
(10, 'Hannah', 10, 795, 'Steak au Poivre', 'French', 'Main Course', 'Red Wine', 'Creme Brulee')


In [12]:
orders_query = """
CREATE PROCEDURE GetMaxQuantity()
BEGIN
    SELECT MAX(quantity) AS 'Max Quantity in Order' FROM orders;
END
"""

cursor.execute(orders_query)


ProgrammingError: 1304 (42000): PROCEDURE GetMaxQuantity already exists

In [13]:
orders_query = """PREPARE GetOrderDetail FROM 
'SELECT OrderID, Quantity, TotalCost FROM Orders WHERE CustomerID = ?'"""
cursor.execute(orders_query)

In [14]:
orders_query = """
CREATE PROCEDURE CancelOrder(IN orderId INT)
BEGIN
    DELETE FROM Orders WHERE OrderID = orderId;
END
"""

cursor.execute(orders_query)


ProgrammingError: 1304 (42000): PROCEDURE CancelOrder already exists

In [15]:
orders_query = """
CREATE PROCEDURE CheckBooking(IN booking_date DATE, IN table_number INT)
BEGIN
    DECLARE bookedTable INT DEFAULT 0;
    
    SELECT COUNT(*)
    INTO bookedTable
    FROM Bookings
    WHERE `Date` = booking_date AND TableNumber = table_number;
    
    IF bookedTable > 0 THEN
        SELECT CONCAT('Table ', table_number, ' is already booked') AS 'Booking status';
    ELSE 
        SELECT CONCAT('Table ', table_number, ' is not booked') AS 'Booking status';
    END IF;
END;
"""

cursor.execute(orders_query)

ProgrammingError: 1304 (42000): PROCEDURE CheckBooking already exists

In [25]:
orders_query = """
CREATE PROCEDURE AddValidBooking(IN bookingDate DATE, IN tableNumber INT, IN customerName VARCHAR(255))
BEGIN
    DECLARE existingBookingCount INT;

    -- Check if the table is already booked
    SELECT COUNT(*) INTO existingBookingCount
    FROM Bookings
    WHERE `Date` = bookingDate AND TableNumber = tableNumber;

    -- Start a transaction
    START TRANSACTION;

    IF existingBookingCount > 0 THEN
        -- The table is already booked, so rollback the transaction
        ROLLBACK;
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Table is already booked for this date.';
    ELSE
        -- The table is available, so insert the new booking
        INSERT INTO Bookings (BookingDate, TableNumber, CustomerName)
        VALUES (bookingDate, tableNumber, customerName);

        -- Commit the transaction
        COMMIT;
    END IF;
END
"""

cursor.execute(orders_query)

ProgrammingError: 1304 (42000): PROCEDURE AddValidBooking already exists

In [26]:
orders_query = """
CREATE PROCEDURE UpdateBooking(IN booking_date DATE, IN booking_id INT)
BEGIN
  UPDATE bookings
  SET `Date` = booking_date
  WHERE BookingID = booking_id;

  SELECT CONCAT('Booking ', booking_id, ' updated') AS Confirmation;
END
"""

cursor.execute(orders_query)

ProgrammingError: 1304 (42000): PROCEDURE UpdateBooking already exists

In [28]:
orders_query = """
CREATE PROCEDURE CancelBooking(IN booking_id_to_cancel INT)
BEGIN
  DELETE FROM Bookings
  WHERE BookingID = booking_id_to_cancel;

  SELECT CONCAT('Booking ', booking_id_to_cancel, ' cancelled') AS Confirmation;
END
"""

cursor.execute(orders_query)

ProgrammingError: 1304 (42000): PROCEDURE CancelBooking already exists